# Chapter 27: Multiprocessing Basics

Python's `multiprocessing` module enables true parallelism by spawning separate operating
system processes, each with its own Python interpreter and memory space. Unlike threads,
processes are not constrained by the Global Interpreter Lock (GIL), making multiprocessing
the go-to approach for CPU-bound workloads.

## Topics Covered
- **Process**: Creating and running child processes
- **Process lifecycle**: `start()`, `join()`, `is_alive()`, exit codes
- **Process identity**: PIDs and process names
- **Pool**: Worker pools for parallel task execution
- **Pool.map()**: Distributing work across processes
- **Pool.apply_async()**: Non-blocking task submission
- **Practical**: Comparing sequential vs parallel execution

## The multiprocessing Module

The `multiprocessing` module mirrors the `threading` API but uses **processes** instead of
threads. Each child process gets its own memory space and Python interpreter, which means:

- No GIL contention -- true parallel execution on multiple CPU cores
- No shared state by default -- data must be explicitly passed or shared
- Higher overhead than threads (process creation is more expensive)
- Functions passed to processes must be **picklable** (defined at module level)

In [ ]:
import multiprocessing
import os
import time

# Basic information about the current environment
print(f"Python process PID: {os.getpid()}")
print(f"CPU count: {os.cpu_count()}")
print(f"Start method: {multiprocessing.get_start_method()}")

## Creating a Process

The `multiprocessing.Process` class creates a new process. You provide a **target** function
and optional **args** or **kwargs**. The process does not start until you call `.start()`.

Key methods:
- `start()` -- spawn the child process and begin execution
- `join(timeout=None)` -- block until the process finishes (or timeout expires)
- `is_alive()` -- check if the process is still running
- `terminate()` / `kill()` -- forcibly stop the process

In [ ]:
import multiprocessing
import os


def worker(name: str) -> None:
    """A simple worker function that runs in a child process."""
    pid: int = os.getpid()
    parent_pid: int = os.getppid()
    print(f"Worker '{name}' running in PID {pid} (parent PID: {parent_pid})")


# Create and start a process
p = multiprocessing.Process(target=worker, args=("alpha",))
print(f"Before start: is_alive={p.is_alive()}, pid={p.pid}")

p.start()
print(f"After start:  is_alive={p.is_alive()}, pid={p.pid}")

p.join()  # Wait for the process to finish
print(f"After join:   is_alive={p.is_alive()}, exitcode={p.exitcode}")

## Process Lifecycle and Exit Codes

Every process has a lifecycle: created, started, running, and terminated. After a process
finishes, you can inspect its `exitcode`:

| Exit code | Meaning |
|-----------|:--------|
| `0` | Normal termination |
| `> 0` | Exception occurred (exit code 1) |
| `< 0` | Killed by signal `-N` (e.g., -9 = SIGKILL) |
| `None` | Process has not yet terminated |

In [ ]:
import multiprocessing
import time


def quick_task() -> None:
    """A task that finishes quickly."""
    time.sleep(0.1)


def slow_task() -> None:
    """A task that takes longer."""
    time.sleep(2.0)


# Demonstrate process lifecycle
p = multiprocessing.Process(target=quick_task, name="QuickWorker")
print(f"Created:  name={p.name}, alive={p.is_alive()}, exitcode={p.exitcode}")

p.start()
print(f"Started:  name={p.name}, alive={p.is_alive()}, exitcode={p.exitcode}")

p.join()
print(f"Finished: name={p.name}, alive={p.is_alive()}, exitcode={p.exitcode}")

# Demonstrate join with timeout
p2 = multiprocessing.Process(target=slow_task, name="SlowWorker")
p2.start()
p2.join(timeout=0.5)  # Wait at most 0.5 seconds
print(f"\nAfter timeout: alive={p2.is_alive()}, exitcode={p2.exitcode}")

p2.terminate()  # Forcibly stop the process
p2.join()       # Clean up
print(f"After terminate: alive={p2.is_alive()}, exitcode={p2.exitcode}")

## Multiple Processes and Separate Memory

Each process has its own memory space. Modifications to variables in a child process do
**not** affect the parent process. This is fundamentally different from threading, where
threads share the same memory.

In [ ]:
import multiprocessing
import os

# Use a Queue to collect results from child processes
# (since child processes have separate memory)


def compute_square(n: int, result_queue: multiprocessing.Queue) -> None:
    """Compute a square and put the result in a queue."""
    pid: int = os.getpid()
    result: int = n * n
    result_queue.put((n, result, pid))


# Create multiple processes
queue: multiprocessing.Queue = multiprocessing.Queue()
processes: list[multiprocessing.Process] = []

for i in range(4):
    p = multiprocessing.Process(target=compute_square, args=(i, queue))
    processes.append(p)
    p.start()

# Wait for all processes to complete
for p in processes:
    p.join()

# Collect results
print(f"Main process PID: {os.getpid()}")
print("\nResults from child processes:")
while not queue.empty():
    n, result, pid = queue.get()
    print(f"  {n}^2 = {result} (computed by PID {pid})")

## Process Names and Daemon Processes

Processes can be named for easier debugging. A **daemon** process runs in the background
and is automatically terminated when the main process exits.

- Set `daemon=True` before calling `start()` to create a daemon process
- Daemon processes cannot spawn child processes of their own
- They are useful for background tasks that should not prevent program exit

In [ ]:
import multiprocessing
import os
import time


def named_worker() -> None:
    """Worker that prints its own name and PID."""
    proc = multiprocessing.current_process()
    print(f"  Process name={proc.name}, PID={os.getpid()}, daemon={proc.daemon}")


# Named processes
p1 = multiprocessing.Process(target=named_worker, name="Worker-A")
p2 = multiprocessing.Process(target=named_worker, name="Worker-B")

p1.start()
p2.start()
p1.join()
p2.join()

# Daemon process example
print("\nDaemon process:")
p3 = multiprocessing.Process(target=named_worker, name="Daemon-Worker", daemon=True)
p3.start()
p3.join(timeout=2)  # Must join daemon processes explicitly if you want to wait
print(f"  p3 daemon={p3.daemon}, exitcode={p3.exitcode}")

## Pool: Managing Worker Processes

Creating individual `Process` objects works for a few tasks, but for many tasks you want a
**pool** of reusable worker processes. `multiprocessing.Pool` manages a fixed number of
worker processes and distributes tasks to them.

Key methods:
- `map(func, iterable)` -- apply `func` to every item, return ordered results (blocking)
- `starmap(func, iterable)` -- like `map()` but unpacks argument tuples
- `apply(func, args)` -- call `func` with `args` in a worker (blocking)
- `apply_async(func, args)` -- non-blocking version, returns `AsyncResult`
- `map_async(func, iterable)` -- non-blocking version of `map()`

In [ ]:
import multiprocessing
import os


def square(x: int) -> int:
    """Compute the square of a number."""
    return x * x


# Pool.map() distributes work across a pool of workers
with multiprocessing.Pool(processes=2) as pool:
    results: list[int] = pool.map(square, [1, 2, 3, 4, 5, 6, 7, 8])

print(f"Input:   {list(range(1, 9))}")
print(f"Squared: {results}")

In [ ]:
import multiprocessing


def power(base: int, exp: int) -> int:
    """Raise base to the given exponent."""
    return base ** exp


# Pool.starmap() unpacks tuples of arguments
args: list[tuple[int, int]] = [(2, 3), (3, 3), (4, 2), (5, 2), (10, 3)]

with multiprocessing.Pool(processes=2) as pool:
    results: list[int] = pool.starmap(power, args)

for (base, exp), result in zip(args, results):
    print(f"  {base}^{exp} = {result}")

## Pool.apply_async(): Non-Blocking Execution

`apply_async()` submits a single task to the pool without blocking. It returns an
`AsyncResult` object that you can use to check status and retrieve results later.

- `result.get(timeout=None)` -- block until the result is ready
- `result.ready()` -- check if the result is available
- `result.successful()` -- check if the task completed without error

In [ ]:
import multiprocessing
import time


def slow_square(x: int) -> int:
    """Compute a square with a simulated delay."""
    time.sleep(0.3)
    return x * x


with multiprocessing.Pool(processes=2) as pool:
    # Submit tasks asynchronously
    async_results = [
        pool.apply_async(slow_square, args=(i,))
        for i in range(1, 6)
    ]

    # Check status while tasks are running
    print("Submitted 5 tasks...")
    time.sleep(0.1)
    for i, ar in enumerate(async_results, 1):
        print(f"  Task {i}: ready={ar.ready()}")

    # Collect results (blocks until ready)
    print("\nCollecting results:")
    for i, ar in enumerate(async_results, 1):
        result: int = ar.get(timeout=5)
        print(f"  {i}^2 = {result}")

## Separate Process IDs

A key characteristic of multiprocessing is that each worker runs in a separate OS process
with its own PID. This is what gives us true parallelism, but it also means that data
must be serialized (pickled) to pass between processes.

In [ ]:
import multiprocessing
import os


def get_pid(_: int = 0) -> int:
    """Return the current process ID."""
    return os.getpid()


main_pid: int = os.getpid()
print(f"Main process PID: {main_pid}")

with multiprocessing.Pool(processes=3) as pool:
    pids: list[int] = pool.map(get_pid, range(6))

print(f"Worker PIDs:      {pids}")
unique_pids: set[int] = set(pids)
print(f"Unique workers:   {unique_pids}")
print(f"\nAll worker PIDs differ from main: {all(pid != main_pid for pid in pids)}")
print(f"Number of unique workers: {len(unique_pids)}")

## Practical: Sequential vs Parallel Execution

Let us compare the execution time of a CPU-bound workload run sequentially versus
in parallel using `Pool.map()`. This demonstrates the real benefit of multiprocessing
for CPU-intensive tasks.

In [ ]:
import multiprocessing
import time


def cpu_bound_task(n: int) -> int:
    """A CPU-bound task: sum of squares up to n."""
    total: int = 0
    for i in range(n):
        total += i * i
    return total


workload: list[int] = [2_000_000] * 8

# Sequential execution
start: float = time.perf_counter()
sequential_results: list[int] = [cpu_bound_task(n) for n in workload]
sequential_time: float = time.perf_counter() - start

# Parallel execution
start = time.perf_counter()
with multiprocessing.Pool(processes=4) as pool:
    parallel_results: list[int] = pool.map(cpu_bound_task, workload)
parallel_time: float = time.perf_counter() - start

print(f"Sequential time: {sequential_time:.3f}s")
print(f"Parallel time:   {parallel_time:.3f}s")
print(f"Speedup:         {sequential_time / parallel_time:.2f}x")
print(f"Results match:   {sequential_results == parallel_results}")

## Error Handling in Processes

When a child process raises an exception, the behavior depends on how you launched it:

- **Process**: The exception is raised in the child only; the parent sees a non-zero `exitcode`
- **Pool.map()**: The exception is re-raised in the parent when results are collected
- **Pool.apply_async()**: The exception is re-raised when you call `.get()` on the result

In [ ]:
import multiprocessing


def risky_task(x: int) -> float:
    """A task that might fail."""
    if x == 0:
        raise ValueError("Cannot process zero!")
    return 100.0 / x


# Error with Pool.map() -- exception propagates to parent
print("Testing Pool.map() error handling:")
try:
    with multiprocessing.Pool(2) as pool:
        pool.map(risky_task, [5, 2, 0, 1])  # 0 will cause an error
except ValueError as e:
    print(f"  Caught in parent: {e}")

# Error with apply_async() -- exception on .get()
print("\nTesting apply_async() error handling:")
with multiprocessing.Pool(2) as pool:
    future = pool.apply_async(risky_task, args=(0,))
    try:
        result = future.get(timeout=5)
    except ValueError as e:
        print(f"  Caught from future.get(): {e}")
    print(f"  future.successful(): {future.successful()}")

## Pool with Initializer

You can pass an `initializer` function to `Pool()` that runs once in each worker process
when it starts. This is useful for setting up per-worker state (database connections,
loading large data, etc.).

In [ ]:
import multiprocessing
import os

# Module-level variable that will be set by the initializer
worker_id: int = -1


def init_worker(base_id: int) -> None:
    """Initialize each worker with a unique ID."""
    global worker_id
    worker_id = base_id + os.getpid() % 100
    print(f"  Worker initialized: PID={os.getpid()}, worker_id={worker_id}")


def task_with_state(x: int) -> str:
    """A task that uses worker-local state."""
    return f"worker_id={worker_id}, PID={os.getpid()}, input={x}, result={x*x}"


print("Initializing pool with per-worker setup:")
with multiprocessing.Pool(processes=2, initializer=init_worker, initargs=(1000,)) as pool:
    results: list[str] = pool.map(task_with_state, range(4))

print("\nResults:")
for r in results:
    print(f"  {r}")

## Summary

### Key Takeaways

| Concept | API | Purpose |
|---------|-----|:--------|
| **Process** | `multiprocessing.Process` | Spawn a single child process |
| **Lifecycle** | `start()`, `join()`, `is_alive()` | Control process execution |
| **Exit codes** | `p.exitcode` | `0`=success, `>0`=error, `<0`=signal |
| **Daemon** | `Process(daemon=True)` | Background process, auto-killed on exit |
| **Pool** | `multiprocessing.Pool(n)` | Manage a pool of `n` worker processes |
| **map()** | `pool.map(func, iterable)` | Distribute work, return ordered results |
| **starmap()** | `pool.starmap(func, args)` | Like map but unpacks argument tuples |
| **apply_async()** | `pool.apply_async(func, args)` | Non-blocking single task submission |
| **Initializer** | `Pool(initializer=func)` | Run setup code once per worker |

### Best Practices
- Always use `Pool` as a context manager (`with` statement) to ensure workers are cleaned up
- Define worker functions at module level so they can be pickled
- Use `join()` to wait for processes and avoid zombie processes
- Prefer `Pool.map()` for batch processing over manually managing `Process` objects
- Handle exceptions from workers -- they propagate when you collect results
- Choose the number of pool workers based on `os.cpu_count()` for CPU-bound tasks